<a href="https://colab.research.google.com/github/arshpreetsingh134/Image_to_speech/blob/main/neural_cap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI
!pip freeze | grep "pycocotools"

  Cloning https://github.com/philferriere/cocoapi.git to /tmp/pip-req-build-u_n2kx_j
  Running command git clone -q https://github.com/philferriere/cocoapi.git /tmp/pip-req-build-u_n2kx_j
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=266599 sha256=beebfcacf168caa1eb2bdaabcb3dc49861ec0861b6cd5da7cfaabee61796a556
  Stored in directory: /tmp/pip-ephem-wheel-cache-4m6kd9ca/wheels/69/2b/12/2fa959e49f73d26cff202c2f4e5079096c9c57c8a8509fd75c
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.2
    Uninstalling pycocotools-2.0.2:
      Successfully uninstalled pycocotools-2.0.2
pycocotools==2.0


In [2]:
!pip install pycocotools

In [3]:
from pycocotools import mask as mask 
print(mask.__author__)

tsungyi


In [4]:
import tensorflow as tf

# We'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import collections
import random
import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

In [6]:
# Downloading & Unzipping captions for Train and Validation Sets

# os.system('wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip')

# os.system('unzip annotations_trainval2014.zip')

# Image Annotations
annot_folder = '/annotations/'
if not os.path.exists(os.path.abspath('.') + annot_folder):
  annot_zip = tf.keras.utils.get_file('train2014.zip',
                                      cache_subdir=os.path.abspath('.'),
                                      origin = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                      extract = True)
  annot_file = os.path.dirname(annot_zip)+'/annotations/captions_train2014.json'
  os.remove(annot_zip)

# Image Files: Train Data 
# image_folder = '/train2014/'
# if not os.path.exists(os.path.abspath('.') + image_folder):
#   annot_zip = tf.keras.utils.get_file('train2014.zip',
#                                       cache_subdir=os.path.abspath('.'),
#                                       origin = 'http://images.cocodataset.org/zips/train2014.zip',
#                                       extract = True)
  
#   PATH = os.path.dirname(image_zip) + image_folder
#   os.remove(image_zip)
# else:
#   PATH = os.path.abspath('.') + image_folder

  643760128/13510573713 [>.............................] - ETA: 6:32

KeyboardInterrupt: ignored

In [7]:
val = json.load(open('annotations/captions_val2014.json', 'r'))
train = json.load(open('annotations/captions_train2014.json', 'r'))

In [8]:
print(val.keys())
print(val['info'])
print(len(val['images']))
print(len(val['annotations']))

dict_keys(['info', 'images', 'licenses', 'annotations'])
{'description': 'COCO 2014 Dataset', 'url': 'http://cocodataset.org', 'version': '1.0', 'year': 2014, 'contributor': 'COCO Consortium', 'date_created': '2017/09/01'}
40504
202654


In [9]:
print(val['images'][0])
print(val['annotations'][0])

{'license': 3, 'file_name': 'COCO_val2014_000000391895.jpg', 'coco_url': 'http://images.cocodataset.org/val2014/COCO_val2014_000000391895.jpg', 'height': 360, 'width': 640, 'date_captured': '2013-11-14 11:18:45', 'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg', 'id': 391895}
{'image_id': 203564, 'id': 37, 'caption': 'A bicycle replica with a clock as the front wheel.'}


In [10]:
# Combine all Images and Captions together
imgs = val['images']+train['images']
caps = val['annotations']+train['annotations']

In [11]:
# Grouping captions by image ID
id_cap = collections.defaultdict(list)
for cap in caps:
  img_id = cap['image_id']
  if img_id not in id_cap:
    id_cap[img_id]=[]
  id_cap[img_id].append(cap)

In [12]:
# Creating the json blob
out = []
for i,img in enumerate(imgs):
    imgid = img['id']
    
    # coco specific here, they store train/val images separately
    loc = 'train2014' if 'train' in img['file_name'] else 'val2014'
    
    jimg = {}
    jimg['file_path'] = os.path.join(loc, img['file_name'])
    jimg['id'] = imgid
    
    sents = []
    annotsi = id_cap[imgid]
    for a in annotsi:
        sents.append(a['caption'])
    jimg['captions'] = sents
    out.append(jimg)
    
json.dump(image_path_to_caption, open('coco_raw.json', 'w'))

In [13]:
print(out[0])

{'file_path': 'val2014/COCO_val2014_000000391895.jpg', 'id': 391895, 'captions': ['A man with a red helmet on a small moped on a dirt road. ', 'Man riding a motor bike on a dirt road on the countryside.', 'A man riding on the back of a motorcycle.', 'A dirt path with a young person on a motor bike rests to the foreground of a verdant area with a bridge and a background of cloud-wreathed mountains. ', 'A man in a red shirt and a red hat is on a motorcycle on a hill side.']}
